In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import copy
from multiprocessing import cpu_count
from typing import List, Iterable, Set, Optional, Dict, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset
from tqdm import tqdm
import numpy as np

from model import SwipeCurveTransformer, get_m1_model, get_m1_bigger_model, get_m1_smaller_model
from tokenizers import CharLevelTokenizerv2, KeyboardTokenizerv1
from tokenizers import ALL_CYRILLIC_LETTERS_ALPHABET_ORD
from dataset import CurveDataset, CurveDatasetSubset
from word_generators import GreedyGenerator, BeamGenerator
from predict import Predictor
from metrics import get_mmr
from aggregate_predictions import (separate_out_vocab_all_crvs,
                                   append_preds,
                                   create_submission,
                                   merge_default_and_extra_preds)
from nearest_key_lookup import ExtendedNearestKeyLookup
from transforms import FullTransform, InitTransform, GetItemTransform

In [1]:
IN_KAGGLE = False

if IN_KAGGLE:
    DATA_ROOT = "/kaggle/input/yandex-cup-playground"
    MODELS_DIR = ""
else:
    DATA_ROOT = "../data/data_separated_grid"
    MODELS_ROOT = "../data/trained_models"

In [3]:
VAL_PATH = os.path.join(DATA_ROOT, "valid__in_train_format.jsonl")
TEST_PATH = os.path.join(DATA_ROOT, "test.jsonl")

VOCAB_PATH = os.path.join(DATA_ROOT, "voc.txt")
G_NAME_TO_GRID_PATH = os.path.join(DATA_ROOT, "gridname_to_grid.json")

In [5]:
def get_grid(grid_name: str, grids_path: str) -> dict:
    with open(grids_path, "r", encoding="utf-8") as f:
        return json.load(f)[grid_name]

In [6]:
def get_gridname_to_out_of_bounds_coords_dict(
        data_paths: List[str], gridname_to_wh: dict,
        totals: Iterable[Optional[int]] = None
        ) -> Dict[str, Set[Tuple[int, int]]]:
    """
    Returns a dictionary with grid names as keys and lists of out of bounds coordinates as values.
    """
    totals = totals or [None] * len(data_paths)
    
    gname_to_out_of_bounds = {gname: set() for gname in gridname_to_wh.keys()}

    for data_path, total in zip(data_paths, totals):
        with open(data_path, "r", encoding="utf-8") as json_file:
            for line in tqdm(json_file, total=total):
                json_data = json.loads(line)
                curve = json_data['curve']
                grid_name = curve['grid_name']
                w, h = gridname_to_wh[grid_name]
                X, Y = curve['x'], curve['y']
                out_of_bounds = set((x, y) for x, y in zip(X, Y) 
                                    if x < 0 or x >= w or y < 0 or y >= h)
                gname_to_out_of_bounds[grid_name].update(out_of_bounds)
    return gname_to_out_of_bounds

In [7]:
def get_datasets(grid_name_to_grid_path: str,
                 data_paths: Iterable[str], totals: Iterable[Optional[int]],
                 nearest_key_candidates: Set[str],
                 kb_tokenizer: KeyboardTokenizerv1,
                 word_char_tokenizer: CharLevelTokenizerv2,
                 is_full_transform: bool,
                 store_gnames: bool
                 ) -> List[CurveDataset]:
    
    grid_names = ('default', 'extra')
    gridname_to_grid  = {gname: get_grid(gname, grid_name_to_grid_path) 
                         for gname in grid_names}

    gname_to_wh = {
        gname: (grid['width'], grid['height']) 
        for gname, grid in gridname_to_grid.items()
    }
    
    print("Accumulating out-of-bounds coordinates...")
    gname_to_out_of_bounds = get_gridname_to_out_of_bounds_coords_dict(
        data_paths, gname_to_wh, totals=totals
    )
    
    print("Creating ExtendedNearestKeyLookups...")
    gridname_to_nkl = {
        gname: ExtendedNearestKeyLookup(grid, nearest_key_candidates, gname_to_out_of_bounds[gname])
        for gname, grid in gridname_to_grid.items()
    }

    if is_full_transform:
        init_transform = FullTransform(
            grid_name_to_nk_lookup=gridname_to_nkl,
            grid_name_to_wh=gname_to_wh,
            kb_tokenizer=kb_tokenizer,
            word_tokenizer=word_char_tokenizer,
            include_time=False,
            include_velocities=True,
            include_accelerations=True
        )

        get_item_transform = None
    
    else:
        init_transform = InitTransform(
            grid_name_to_nk_lookup=gridname_to_nkl,
            kb_tokenizer=kb_tokenizer,
        )

        get_item_transform = GetItemTransform(
            grid_name_to_wh=gname_to_wh,
            word_tokenizer=word_char_tokenizer,
            include_time=False,
            include_velocities=True,
            include_accelerations=True,
        )
    
    print("Creating datasets...")
    datasets = []
    for d_path, total in zip(data_paths, totals):
        ds = CurveDataset(
            data_path=d_path,
            store_gnames = store_gnames,
            init_transform=init_transform,
            get_item_transform=get_item_transform,
            total = total,
        )
        datasets.append(ds)
    
    return datasets

In [8]:
kb_tokenizer = KeyboardTokenizerv1()
word_char_tokenizer = CharLevelTokenizerv2(VOCAB_PATH)

data_paths = [
    VAL_PATH,
    TEST_PATH
]
totals = [10_000, 10_000]


# train_dataset, val_dataset = get_datasets(
val_dataset, test_dataset = get_datasets(
    grid_name_to_grid_path=G_NAME_TO_GRID_PATH,
    data_paths = data_paths, totals = totals,
    nearest_key_candidates = ALL_CYRILLIC_LETTERS_ALPHABET_ORD,
    kb_tokenizer=kb_tokenizer,
    word_char_tokenizer=word_char_tokenizer,
    is_full_transform = True,
    store_gnames = True
)

Accumulating out-of-bounds coordinates...


100%|██████████| 10000/10000 [00:00<00:00, 18743.27it/s]


Creating ExtendedNearestKeyLookups...
Creating datasets...


100%|██████████| 10000/10000 [00:10<00:00, 984.38it/s]


In [9]:
val_default_dataset = CurveDatasetSubset(val_dataset, "default")
val_extra_dataset = CurveDatasetSubset(val_dataset, "extra")

test_default_dataset = CurveDatasetSubset(test_dataset, "default")
test_extra_dataset = CurveDatasetSubset(test_dataset, "extra")

In [10]:
def get_targets(dataset: CurveDataset) -> List[str]:
    targets = []
    for _, target_tokens in dataset:
        target = word_char_tokenizer.decode(target_tokens[:-1])
        targets.append(target)
    return targets

In [11]:
def get_vocab_set(vocab_path: str):
    with open(vocab_path, 'r', encoding = "utf-8") as f:
        return set(f.read().splitlines())

In [12]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [13]:
device = torch.device('cpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
grid_name = "default"
model_getter = get_m1_bigger_model
weights_path = os.path.join(MODELS_ROOT, "m1_bigger/m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, word_char_tokenizer, device)}

c:\Users\proshian\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [15]:
greedy_generator = GreedyGenerator(model, word_char_tokenizer, device)


print("{:<20} {:<20} {:<20}".format("target", "prediction", "prob"))
print("-"*49)

n_examples = 30

for i, data in enumerate(val_default_dataset):

    (xyt, kb_tokens, dec_in_char_seq), target = data

    score, pred = greedy_generator(xyt, kb_tokens)[0]

    target = word_char_tokenizer.decode(target[:-1])
    print("{:<20} {:<20} {:<20.5f}".format(target, pred, np.exp(score)))

    if i >= n_examples:
        break

target               prediction           prob                
-------------------------------------------------
на                   на                   0.99890             
все                  все                  0.99983             
добрый               добрый               0.99704             
девочка              девочка              0.99489             
сказала              сказала              0.99733             
скинь                скинь                0.99880             
геев                 геев                 0.65820             
тобой                тобой                0.99947             
была                 бысса                0.28788             
да                   да                   0.99748             
муж                  муж                  0.91975             
щас                  щас                  0.99749             
она                  она                  0.99797             
проблема             проблема             0.98953             
билай

In [16]:
beam_generator = BeamGenerator(model, word_char_tokenizer, device)

print("{:<20} {:<20} {:<20}".format("target", "prediction", "prob"))
print("-"*49)

n_examples = 10

for i, data in enumerate(val_default_dataset):

    normalization_factor = 0.5

    (xyt, kb_tokens, dec_in_char_seq), target = data

    score, pred = beam_generator(xyt, kb_tokens, max_steps_n = 35, 
        normalization_factor=normalization_factor)[0]

    target = word_char_tokenizer.decode(target[:-1])

    denorm_score = score * (len(pred) + 1)**normalization_factor

    print("{:<20} {:<20} {:<20.5f}".format(target, pred, np.exp(denorm_score)))

    if i >= n_examples:
        break

target               prediction           prob                
-------------------------------------------------


c:\Users\proshian\Documents\yandex-cup-2023-ml-neuroswipe\src\word_generators.py:115: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  next_tokens_logproba = F.log_softmax(next_tokens_logits)


на                   на                   1.00096             
все                  все                  1.00015             
добрый               добрый               1.00278             
девочка              девочка              1.00484             
сказала              сказала              1.00253             
скинь                скинь                1.00111             
геев                 геев                 1.46492             
тобой                тобой                1.00049             
была                 бысса                3.16716             
да                   да                   1.00219             
муж                  муж                  1.07770             


Вероятности больше 1... Что-то в beamsearch напутано с нормализацией, потому что после ее устранения все равно больше 1

In [17]:
val_default_targets = get_targets(val_default_dataset)
val_extra_targets = get_targets(val_extra_dataset)

# Evaluate models separately and as a pair

In [42]:
print(cpu_count())

4


In [16]:
val_default_targets = get_targets(val_default_dataset)
val_extra_targets = get_targets(val_extra_dataset)

In [ ]:
default_predictions = weights_to_raw_predictions(
    grid_name = "default",
    model_getter=get_m1_bigger_model,
    weights_path = os.path.join(MODELS_ROOT, "m1_bigger/m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt"),
    word_char_tokenizer=word_char_tokenizer,
    dataset=val_default_dataset,
    generator_ctor=GreedyGenerator,
    n_workers=4
)

In [135]:
default_MMR =  get_mmr(default_predictions, val_default_targets)
default_MMR

0.8531223449447749

In [136]:
default_predictions_best_bigger = default_predictions

In [137]:
default_predictions_best_bigger_clean, _ = separate_out_vocab_all_crvs(default_predictions_best_bigger, vocab_set)

In [138]:
sum(bool(el) for el in default_predictions_best_bigger_clean)

8784

In [182]:
default_predictions = weights_to_raw_predictions(
    grid_name = "default",
    model_getter=get_m1_smaller_model,
    weights_path = os.path.join(MODELS_ROOT, "m1_smaller/m1_smaller_v2_2023_11_11_17_43_35_0_33179_default_l2_1e_05_ls0_02.pt"),
    word_char_tokenizer=word_char_tokenizer,
    dataset=val_default_dataset,
    generator_ctor=GreedyGenerator,
    n_workers=4
)

100%|██████████| 9416/9416 [04:08<00:00, 37.90it/s]


In [183]:
default_predictions_clean, _ = separate_out_vocab_all_crvs(default_predictions, vocab_set)

In [184]:
sum(bool(el) for el in default_predictions_clean)

8449

In [185]:
default_MMR =  get_mmr(default_predictions, val_default_targets)
default_MMR

0.8221112999150383

In [110]:
# grid_name = "default"
# model_getter = get_m1_bigger_model
# weights_path = os.path.join(MODELS_ROOT, "m1_bigger/m1_bigger_v2__2023_11_11__13_17_50__0.13845_default_l2_0_ls0_switch_0.pt")
# model = model_getter(device, weights_path)
# grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, word_char_tokenizer, device)}

In [111]:
# default_predictions = predict_greedy_raw_multiproc(val_default_dataset,
#                                                     grid_name_to_greedy_generator,
#                                                     num_workers=4)

100%|██████████| 9416/9416 [05:02<00:00, 31.10it/s]


In [29]:
grid_name = "extra"
model_getter = get_m1_model
weights_path = os.path.join(MODELS_ROOT, "m1_v2/m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, word_char_tokenizer, device)}

In [31]:
extra_predictions = predict_raw_mp(val_extra_dataset,
                                   grid_name_to_greedy_generator,
                                   num_workers=4)

100%|██████████| 584/584 [00:18<00:00, 31.60it/s]


In [33]:
extra_MMR = get_mmr(extra_predictions, val_extra_targets)
extra_MMR

0.851027397260274

In [41]:
all_preds = merge_default_and_extra_preds(default_predictions, extra_predictions, val_default_dataset.grid_name_idxs, val_extra_dataset.grid_name_idxs)

In [40]:
all_targets = None
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding='utf-8') as f:
    all_targets = f.read().splitlines() 

In [46]:
full_MMR = get_mmr(all_preds, all_targets)
full_MMR

0.8512

In [365]:
max(len(el) for el in all_targets)

19

In [ ]:
{"m1_v2/best_model__2023_11_09__10_36_02__0.14229_default_switch_2_try_2.pt": 0.8512107051826678,
 "m1_v2/m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt": 0.851027397260274,
 
 "m1_bigger/m1_bigger_v2__2023_11_10__13_38_32__0.50552_default_l2_5e-05_ls0.045_switch_0.pt": 0.810429056924384,
 "m1_bigger/m1_bigger_v2__2023_11_10__16_36_38__0.49848_default_l2_5e-05_ls0.045_switch_0.pt": 0.818500424808836,
 "m1_bigger/m1_bigger_v2__2023_11_10__21_51_01__0.49382_default_l2_5e-05_ls0.045_switch_0.pt": 0.8210492778249787,
 
 "m1_bigger/m1_bigger_v2__2023_11_11__13_17_50__0.13845_default_l2_0_ls0_switch_0.pt": 0.8512107051826678,
 "m1_bigger/m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt": 0.8531223449447749,
 
 "m1_smaller/m1_smaller_v2_2023_11_11_17_43_35_0_33179_default_l2_1e_05_ls0_02.pt": 0.8221112999150383}

# Let's create a greedy submission

In [73]:
from typing import Set, List, Tuple

In [ ]:
grid_name = "default"
model_getter = get_m1_model
weights_path = os.path.join(MODELS_ROOT, "m1_v2/m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, word_char_tokenizer, device)}
default_test_predictions = predict_raw_mp(test_default_dataset,
                                          grid_name_to_greedy_generator,
                                          num_workers=3)

In [15]:
grid_name = "extra"
model_getter = get_m1_model
weights_path = os.path.join(MODELS_ROOT, "m1_v2/m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt")
model = model_getter(device, weights_path)
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, word_char_tokenizer, device)}
extra_test_predictions = predict_raw_mp(test_extra_dataset,
                                        grid_name_to_greedy_generator,
                                        num_workers=3)

100%|██████████| 627/627 [00:20<00:00, 30.18it/s]


In [66]:
all_test_preds = merge_default_and_extra_preds(default_test_predictions, extra_test_predictions, test_default_dataset.grid_name_idxs, test_extra_dataset.grid_name_idxs)

In [69]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [91]:
clean_test_preds, invalid_test_preds = separate_out_vocab_all_crvs(all_test_preds, vocab_set)

In [92]:
clean_test_preds[:10]

[['на'],
 ['что'],
 ['опоздания'],
 ['сколько'],
 [],
 ['не'],
 ['как'],
 ['садовод'],
 ['заметил'],
 ['ваги']]

In [81]:
augment_list = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    augment_lines = f.read().splitlines()
augment_list = [line.split(",") for line in augment_lines]

In [96]:
augmented_test_preds = append_preds(clean_test_preds, augment_list, limit = 4)

In [97]:
augmented_test_preds[:10]

[['на', 'неа', 'на', 'ненка'],
 ['что', 'часто', 'частого', 'чисто'],
 ['опоздания', 'опоздания', 'опозданиям', 'оприходования'],
 ['сколько', 'сколько', 'сокольского', 'свердловского'],
 ['дремать', 'дописать', 'донимать', 'дюрренматт'],
 ['не', 'неук', 'нк', 'ненка'],
 ['как', 'как', 'капак', 'капе'],
 ['садовод', 'спародировал', 'садовод', 'сурдоперевод'],
 ['заметил', 'знаменито', 'знаменитого', 'замерил'],
 ['ваги', 'ваенги', 'венгрии', 'ванги']]

In [98]:
submission_name = "m1_v2__0.14229_deault__0.14301_extra__greedy.csv"
out_path = rf"..\data\submissions\{submission_name}"
create_submission(augmented_test_preds, out_path)

# BeamSearch

In [ ]:
from typing import Tuple, List
def remove_beamsearch_probs(preds: List[List[Tuple[float, str]]]) -> List[List[str]]:
    new_preds = []
    for pred_line in preds:
        new_preds_line = []
        for _, word in pred_line:
            new_preds_line.append(word)
        new_preds.append(new_preds_line)
    return new_preds

In [ ]:
def patch_wrong_prediction_shape(prediciton):
    return [pred_el[0] for pred_el in prediciton]

## Beamsearch Evaluation

In [372]:
MAX_VAL_WORD_LEN = max(len(el) for el in all_targets)

generator_kwargs = {
    'max_steps_n': MAX_VAL_WORD_LEN+1,
    'return_hypotheses_n': 7,
    'beamsize': 6,
    'normalization_factor': 0.5,
    'verbose': False
}

In [373]:
grid_name_to_val_dataset = {
    'default': val_default_dataset,
    'extra': val_extra_dataset
}

In [374]:
import pickle

bs_params = [
    ("default", get_m1_bigger_model, "m1_bigger/m1_bigger_v2__2023_11_12__14_51_49__0.13115__greed_acc_0.86034__default_l2_0_ls0_switch_2.pt"),
]


for grid_name, model_getter, weights_f_name in bs_params:

    bs_preds_path = os.path.join("../data/saved_beamsearch_validation_results/",
                                f"{weights_f_name.replace('/', '__')}.pkl")
    
    if os.path.exists(bs_preds_path):
        print(f"Path {bs_preds_path} exists. Skipping.")
        continue

    bs_predictions = weights_to_raw_predictions(
        grid_name = grid_name,
        model_getter=model_getter,
        weights_path = os.path.join(MODELS_ROOT, weights_f_name),
        word_char_tokenizer=word_char_tokenizer,
        dataset=grid_name_to_val_dataset[grid_name],
        generator_ctor=BeamGenerator,
        n_workers=4,
        generator_kwargs=generator_kwargs
    )

    with open(bs_preds_path, 'wb') as f:
        pickle.dump(bs_predictions, f, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 9416/9416 [41:23<00:00,  3.79it/s] 


In [375]:
preds_name = "m1_bigger__m1_bigger_v2__2023_11_12__14_51_49__0.13115__greed_acc_0.86034__default_l2_0_ls0_switch_2.pt.pkl"
bs_preds_path = os.path.join("../data/saved_beamsearch_validation_results/",
                                preds_name)
with open(bs_preds_path, 'rb') as f:
    default_valid_preds_bs = pickle.load(f)

default_valid_preds_bs = patch_wrong_prediction_shape(default_valid_preds_bs)
default_valid_preds_bs = remove_beamsearch_probs(default_valid_preds_bs)
default_valid_preds_bs, _ = separate_out_vocab_all_crvs(default_valid_preds_bs, vocab_set)
get_mmr(default_valid_preds_bs, val_default_targets)

0.8929800339847141

In [340]:
preds_name = "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl"
bs_preds_path = os.path.join("../data/saved_beamsearch_validation_results/",
                                preds_name)
with open(bs_preds_path, 'rb') as f:
    extra_valid_preds_bs = pickle.load(f)

extra_valid_preds_bs = patch_wrong_prediction_shape(extra_valid_preds_bs)
extra_valid_preds_bs = remove_beamsearch_probs(extra_valid_preds_bs)
extra_valid_preds_bs, _ = separate_out_vocab_all_crvs(extra_valid_preds_bs, vocab_set)
get_mmr(extra_valid_preds_bs, val_extra_targets)

0.8876027397260277

In [ ]:
# {
#     "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl": 0.8811533559898118,
#     "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl": 0.8835960067969484,
#     "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl": 0.8900881478334822,
#     "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl": 0.8871590909090984,
#     "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl": 0.887674171622777,
#     "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl": 0.8877740016992428,
    
    
#     "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl": 0.8864383561643838,
#     "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl": 0.8876027397260277
    
# }

In [37]:
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_12__14_51_49__0.13115__greed_acc_0.86034__default_l2_0_ls0_switch_2.pt.pkl", #: 0.8929800339847141,
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl", #: 0.8914698385726496,
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",  #: 0.8900881478334822,
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",  #: E 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",  #: 0.887674171622777,
        # "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",  #: 0.8871590909090984,
        "m1_smaller__m1_smaller_v2__2023_11_12__17_40_42__0.30909_default_l2_1e-05_ls0.02_switch_0.pt.pkl",  # 0.8849384027187835
        # "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",  #: 0.8835960067969484,
        # "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",  #: 0.8811533559898118,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}

# Отранжированы по качесту beamsearch на валидации

In [ ]:
import pickle

default_idxs = val_default_dataset.grid_name_idxs
extra_idxs = val_extra_dataset.grid_name_idxs 

grid_name_to_augmented_preds = {}

for grid_name in ('default', 'extra'):
    bs_pred_list = []

    for f_name in grid_name_to_ranged_bs_model_preds_paths[grid_name]:
        f_path = os.path.join("../data/saved_beamsearch_validation_results/", f_name)
        with open(f_path, 'rb') as f:
            bs_pred_list.append(pickle.load(f))
        
    bs_pred_list = [patch_wrong_prediction_shape(bs_preds) for bs_preds in bs_pred_list] 
    bs_pred_list = [remove_beamsearch_probs(bs_preds) for bs_preds in bs_pred_list]
    bs_pred_list = [separate_out_vocab_all_crvs(bs_preds, vocab_set)[0] for bs_preds in bs_pred_list]


    augmented_preds = bs_pred_list.pop(0)

    while bs_pred_list:
        augmented_preds = append_preds(augmented_preds, bs_pred_list.pop(0))

    grid_name_to_augmented_preds[grid_name] = augmented_preds


full_preds = merge_default_and_extra_preds(
    grid_name_to_augmented_preds['default'],
    grid_name_to_augmented_preds['extra'],
    default_idxs,
    extra_idxs)

In [ ]:
full_preds[:10]

In [ ]:
from collections import defaultdict

n_preds_in_line_dict = defaultdict(int)

for line in full_preds:
    n_preds_in_line_dict[len(line)] += 1

print(n_preds_in_line_dict)

In [ ]:
all_targets = None
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding='utf-8') as f:
    all_targets = f.read().splitlines() 

In [ ]:
get_mmr(full_preds, all_targets)

```python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",#: 0.8900881478334822,
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",#: 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",#: 0.887674171622777,
        "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",#: 0.8871590909090984,
        "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8835960067969484,
        "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8811533559898118,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}
```

0.8936010000000082


``` python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",#: 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",#: 0.8900881478334822,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",#: 0.887674171622777,
        "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",#: 0.8871590909090984,
        "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8835960067969484,
        "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8811533559898118,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}
```

0.892801000000009


``` python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",#: 0.8900881478334822,
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",#: 0.8877740016992428,
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",#: 0.887674171622777,
        "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8835960067969484,
        "m1_smaller__m1_smaller_v2_2023_11_12_01_21_45_0_31891_default_l2_1e_05_ls0_02.pt.pkl",#: 0.8811533559898118,
        "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl",#: 0.8871590909090984,
        ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl"
        ]
}

# должны ранжироваться по качесту beamsearch на валидации
```

0.8936000000000084

## Beamsearch test

In [343]:
MAX_WORD_LEN = 36

generator_kwargs = {
    'max_steps_n': MAX_WORD_LEN - 1,
    'return_hypotheses_n': 7,
    'beamsize': 6,
    'normalization_factor': 0.5,
    'verbose': False
}

In [344]:
grid_name_to_test_dataset = {
    'default': test_default_dataset,
    'extra': test_extra_dataset
}

In [356]:
import pickle

bs_params = [
    ("extra", get_m1_bigger_model, "m1_bigger/m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt"),

    # "m1_smaller__m1_smaller_v2_2023_11_12_08_17_33_0_31223_default_l2_1e_05_ls0_02.pt.pkl": 0.8835960067969484,
    # "m1_bigger__m1_bigger_v2__2023_11_11__15_53_07__0.13636_default_l2_0_ls0_switch_0.pt.pkl": 0.8871590909090984,
    # "m1_bigger__m1_bigger_v2__2023_11_11__16_45_33__0.13721_extra_l2_0_ls0_switch_0.pt.pkl": 0.8864383561643838,
]


for grid_name, model_getter, weights_f_name in bs_params:

    bs_preds_path = os.path.join("../data/saved_beamsearch_results/",
                                f"{weights_f_name.replace('/', '__')}.pkl")
    
    if os.path.exists(bs_preds_path):
        print(f"Path {bs_preds_path} exists. Skipping.")
        continue

    bs_predictions = weights_to_raw_predictions(
        grid_name = grid_name,
        model_getter=model_getter,
        weights_path = os.path.join(MODELS_ROOT, weights_f_name),
        word_char_tokenizer=word_char_tokenizer,
        dataset=grid_name_to_test_dataset[grid_name],
        generator_ctor=BeamGenerator,
        n_workers=4,
        generator_kwargs=generator_kwargs
    )

    with open(bs_preds_path, 'wb') as f:
        pickle.dump(bs_predictions, f, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 627/627 [06:02<00:00,  1.73it/s]


In [161]:
# len(clean_default_test_predictions), sum(bool(el) for el in clean_default_test_predictions)

(9373, 9193)

In [78]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [169]:
# from collections import defaultdict

# n_preds_in_line_dict = defaultdict(int)

# for line in clean_test_predictions:
#     n_preds_in_line_dict[len(line)] += 1

# print(n_preds_in_line_dict)

defaultdict(<class 'int'>, {4: 7404, 3: 1032, 2: 840, 1: 577, 0: 147})


In [170]:
old_preds_path = os.path.join(DATA_ROOT, "test_raw_pred___best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt__best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt.pkl")
with open(old_preds_path, 'rb') as f:
    old_preds_list = pickle.load(f)

In [171]:
old_preds_list = remove_beamsearch_probs(old_preds_list)

In [172]:
old_preds_list_valid, old_preds_list_invalid = separate_out_vocab_all_crvs(old_preds_list, vocab_set)

In [176]:
# submission_name = "default__m1_bigger_13679__m1_v2__14229___extra__14301___with_baseline__beam.csv"
# out_path = rf"..\data\submissions\{submission_name}"
# create_submission(clean_test_baseline_augmented, out_path)

# Submission creation

id 1

```
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': ["m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl"]
}
```

id 2

```
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt.pkl"
    ]
}

# должны ранжироваться по качесту beamsearch на валидации
```

id 3

```python
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger_m1_bigger_v2_2023_11_12_14_51_49_0_13115_greed_acc_0_86034.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt.pkl"
    ]
}

# должны ранжироваться по качесту beamsearch на валидации
```

In [385]:
grid_name_to_ranged_bs_model_preds_paths = {
    'default': [
        "m1_bigger_m1_bigger_v2_2023_11_12_14_51_49_0_13115_greed_acc_0_86034.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__22_18_35__0.13542_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_v2__m1_v2__2023_11_09__10_36_02__0.14229_default_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__00_39_33__0.13297_default_l2_0_ls0_switch_1.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_11__14_29_37__0.13679_default_l2_0_ls0_switch_0.pt.pkl",
        
    ],
    'extra': [
        "m1_v2__m1_v2__2023_11_09__17_47_40__0.14301_extra_l2_1e-05_switch_0.pt.pkl",
        "m1_bigger__m1_bigger_v2__2023_11_12__02_27_14__0.13413_extra_l2_0_ls0_switch_1.pt.pkl"
    ]
}

# должны ранжироваться по качесту beamsearch на валидации

In [386]:
default_idxs = test_default_dataset.grid_name_idxs
extra_idxs = test_extra_dataset.grid_name_idxs 

grid_name_to_augmented_preds = {}

for grid_name in ('default', 'extra'):
    bs_pred_list = []

    for f_name in grid_name_to_ranged_bs_model_preds_paths[grid_name]:
        f_path = os.path.join("../data/saved_beamsearch_results/", f_name)
        with open(f_path, 'rb') as f:
            bs_pred_list.append(pickle.load(f))
        
    bs_pred_list = [patch_wrong_prediction_shape(bs_preds) for bs_preds in bs_pred_list] 
    bs_pred_list = [remove_beamsearch_probs(bs_preds) for bs_preds in bs_pred_list]
    bs_pred_list = [separate_out_vocab_all_crvs(bs_preds, vocab_set)[0] for bs_preds in bs_pred_list]


    augmented_preds = bs_pred_list.pop(0)

    while bs_pred_list:
        augmented_preds = append_preds(augmented_preds, bs_pred_list.pop(0))

    grid_name_to_augmented_preds[grid_name] = augmented_preds


full_preds = merge_default_and_extra_preds(
    grid_name_to_augmented_preds['default'],
    grid_name_to_augmented_preds['extra'],
    default_idxs,
    extra_idxs)

In [387]:
from collections import defaultdict

n_preds_in_line_dict = defaultdict(int)

for line in full_preds:
    n_preds_in_line_dict[len(line)] += 1

print(n_preds_in_line_dict)

defaultdict(<class 'int'>, {4: 8188, 3: 706, 2: 606, 1: 407, 0: 93})


In [388]:
baseline_preds = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    baseline_preds = f.read().splitlines()
baseline_preds = [line.split(",") for line in augment_lines]

In [389]:
full_preds = append_preds(full_preds, baseline_preds)

In [390]:
create_submission(full_preds,
                  f"../data/submissions/id3_with_baseline_without_old_preds")

In [363]:
full_preds_augmentations = [
    
]